# Web Scraping en páginas académicas

En este notebook aprenderemos a extraer información de dos fuentes académicas muy populares:

- **arXiv.org** → repositorio de artículos científicos de acceso abierto
- **Semantic Scholar** → buscador académico con API pública

Veremos **tres enfoques diferentes** para obtener datos:

| Enfoque | Fuente | Herramienta |
|---------|--------|-------------|
| Scraping HTML | Página de artículo en arXiv | `bs4` + `requests` |
| Parsing XML | API de arXiv | `bs4` + `requests` |
| Consumo de API REST | API de Semantic Scholar | `requests` + JSON |

**Librerías necesarias:** `requests`, `beautifulsoup4`, `pandas`

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

---
## Parte 1: Scraping HTML en arXiv

### ¿Qué es arXiv?

**arXiv.org** es un repositorio de acceso abierto con más de 2 millones de artículos de física, matemáticas, informática, biología, etc. Los investigadores publican aquí sus trabajos antes (o en paralelo) de la revisión por pares.

### Objetivo
Vamos a acceder a la **página de un artículo** en arXiv y extraer:
- Título
- Autores
- Resumen (abstract)
- Categorías
- Fecha de publicación
- Enlace al PDF

### 1.1 Descargar la página de un artículo

Cada artículo en arXiv tiene una URL tipo `https://arxiv.org/abs/XXXX.XXXXX`.  
Vamos a usar un artículo famoso sobre **Attention** (la base de los Transformers).

In [ ]:
# URL del artículo "Attention Is All You Need"
url = "https://arxiv.org/abs/1706.03762"

# Hacemos la petición HTTP
respuesta = requests.get(url)
print(f"Status code: {respuesta.status_code}")
print(f"Tamaño de la respuesta: {len(respuesta.text):,} caracteres")

In [ ]:
# Parseamos el HTML con BeautifulSoup
soup = BeautifulSoup(respuesta.text, "html.parser")

# Veamos el título de la página para confirmar que todo va bien
print(soup.title.text.strip())

### 1.2 Extraer el título del artículo

Si inspeccionamos el HTML de arXiv (botón derecho → "Inspeccionar"), veremos que el título del paper está dentro de una etiqueta `<h1 class="title mathjax">`.

In [ ]:
# El título está en un <h1> con clase "title mathjax"
titulo_tag = soup.find("h1", class_="title mathjax")
print("Tag completo:", titulo_tag)
print()

# El tag contiene "Title:" como texto previo, lo quitamos
titulo = titulo_tag.text.replace("Title:", "").strip()
print("Título limpio:", titulo)

### 1.3 Extraer los autores

In [ ]:
# Los autores están en <div class="authors">
autores_div = soup.find("div", class_="authors")

# Cada autor es un enlace <a> dentro de ese div
autores = [a.text.strip() for a in autores_div.find_all("a")]
print(f"Número de autores: {len(autores)}")
print("Autores:", ", ".join(autores))

### 1.4 Extraer el abstract

In [ ]:
# El abstract está en <blockquote class="abstract mathjax">
abstract_tag = soup.find("blockquote", class_="abstract mathjax")

# Quitamos el prefijo "Abstract:" y limpiamos espacios
abstract = abstract_tag.text.replace("Abstract:", "").strip()
print(abstract[:300], "...")

### 1.5 Extraer categorías, fecha y enlace al PDF

In [ ]:
# Categorías: están en <span class="primary-subject">
categoria = soup.find("span", class_="primary-subject").text.strip()
print("Categoría principal:", categoria)

# Fecha: está en el div "submission-history"
historial = soup.find("div", class_="submission-history")
# La primera fecha suele ser la de envío original
fecha_texto = historial.text.strip()
print("\nHistorial de envío (primeras líneas):")
for linea in fecha_texto.split("\n")[:5]:
    if linea.strip():
        print(" ", linea.strip())

# Enlace al PDF: se construye cambiando /abs/ por /pdf/
pdf_url = url.replace("/abs/", "/pdf/")
print(f"\nEnlace al PDF: {pdf_url}")

### 1.6 Extraer varios artículos y crear un DataFrame

Ahora vamos a hacer lo mismo con **varios artículos** clásicos de Deep Learning y guardar los resultados en un DataFrame de pandas.

> **Buenas prácticas:** Entre petición y petición añadimos un `time.sleep()` para no saturar el servidor. arXiv pide un mínimo de 3 segundos entre peticiones.

In [ ]:
# Lista de artículos famosos de Deep Learning (sus IDs en arXiv)
articulos_ids = [
    "1706.03762",  # Attention Is All You Need
    "1810.04805",  # BERT
    "2005.14165",  # GPT-3
    "1512.03385",  # ResNet
    "1406.2661",  # GANs
]

resultados = []

for arxiv_id in articulos_ids:
    url = f"https://arxiv.org/abs/{arxiv_id}"
    print(f"Descargando: {arxiv_id}...", end=" ")
    
    respuesta = requests.get(url)
    
    if respuesta.status_code != 200:
        print(f"ERROR ({respuesta.status_code})")
        continue
    
    soup = BeautifulSoup(respuesta.text, "html.parser")
    
    # Extraemos los campos
    titulo = soup.find("h1", class_="title mathjax").text.replace("Title:", "").strip()
    autores = [a.text.strip() for a in soup.find("div", class_="authors").find_all("a")]
    abstract = soup.find("blockquote", class_="abstract mathjax").text.replace("Abstract:", "").strip()
    categoria = soup.find("span", class_="primary-subject").text.strip()
    
    resultados.append({
        "arxiv_id": arxiv_id,
        "titulo": titulo,
        "autores": ", ".join(autores),
        "num_autores": len(autores),
        "abstract": abstract[:200] + "...",
        "categoria": categoria,
        "url_pdf": f"https://arxiv.org/pdf/{arxiv_id}",
    })
    
    print(f"OK → {titulo[:50]}...")
    time.sleep(3)  # Respetar el rate limit de arXiv

print(f"\n{len(resultados)} artículos extraídos")

In [ ]:
# Creamos el DataFrame
df_arxiv = pd.DataFrame(resultados)
df_arxiv

---
## Parte 2: API XML de arXiv

arXiv ofrece una **API oficial** que devuelve resultados en formato **XML (Atom)**. Es la forma "correcta" de hacer búsquedas masivas.

**Ventajas frente al scraping HTML:**
- No dependemos de que cambien el diseño de la web
- Datos más estructurados y limpios
- Permitido explícitamente por arXiv

**¡Y podemos seguir usando BeautifulSoup!** porque bs4 también parsea XML.

Documentación: https://info.arxiv.org/help/api/index.html

### 2.1 Buscar artículos por tema

La URL de la API tiene esta estructura:
```
http://export.arxiv.org/api/query?search_query=TERMINO&start=0&max_results=10
```

In [ ]:
# Buscamos artículos sobre "large language models"
termino = "large language models"
max_resultados = 4

url_api = f"http://export.arxiv.org/api/query?search_query=all:{termino}&start=0&max_results={max_resultados}"
print(f"URL de la API: {url_api}")

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}

respuesta = requests.get(url_api)
print(f"Status code: {respuesta.status_code}")
print(f"Tipo de contenido: {respuesta.headers.get('Content-Type', 'desconocido')}")

In [ ]:
# Veamos un trozo del XML que nos devuelve
print(respuesta.text[:1000])

### 2.2 Parsear el XML con BeautifulSoup

El XML devuelto sigue el formato **Atom**. Cada artículo está dentro de una etiqueta `<entry>` con sub-etiquetas como `<title>`, `<summary>`, `<author>`, etc.

> **Nota:** Usamos el parser `"xml"` en lugar de `"html.parser"`. Para que funcione necesitamos tener instalado `lxml` (`pip install lxml`).

In [ ]:
# Parseamos el XML
soup_xml = BeautifulSoup(respuesta.text, "xml")

# Cada artículo es un <entry>
entradas = soup_xml.find_all("entry")
print(f"Artículos encontrados: {len(entradas)}")

In [ ]:
# Veamos la estructura de la primera entrada
primera = entradas[1]
print(primera.prettify()[:1500])

In [ ]:
# Extraemos los datos de cada entrada
resultados_api = []

for entry in entradas:
    # ID de arXiv (viene como URL, extraemos solo el ID)
    arxiv_url = entry.find("id").text
    arxiv_id = arxiv_url.split("/abs/")[-1]
    
    # Título (a veces viene con saltos de línea, los limpiamos)
    titulo = entry.find("title").text.strip().replace("\n", " ")
    
    # Autores: cada uno está en su propio <author><name>...</name></author>
    autores = [author.find("name").text for author in entry.find_all("author")]
    
    # Resumen
    resumen = entry.find("summary").text.strip().replace("\n", " ")
    
    # Fecha de publicación
    fecha = entry.find("published").text[:10]  # Solo YYYY-MM-DD
    
    # Categoría principal
    categoria = entry.find("category")["term"]
    
    # Enlace al PDF
    enlaces = entry.find_all("link")
    pdf_link = ""
    for enlace in enlaces:
        if enlace.get("title") == "pdf":
            pdf_link = enlace["href"]
            break
    
    resultados_api.append({
        "arxiv_id": arxiv_id,
        "titulo": titulo,
        "primer_autor": autores[0] if autores else "",
        "num_autores": len(autores),
        "fecha": fecha,
        "categoria": categoria,
        "resumen": resumen[:150] + "...",
        "pdf": pdf_link,
    })

print(f" {len(resultados_api)} artículos procesados")

In [ ]:
df_arxiv_api = pd.DataFrame(resultados_api)
df_arxiv_api

### 2.3 Ejemplo: cambiar el término de búsqueda

Puedes probar con otros temas. Algunos ejemplos:
- `"web scraping"` 
- `"natural language processing"`
- `"climate change prediction"`
- `"quantum computing"`

In [ ]:
# Prueba: cambia el término aquí y ejecuta esta celda
termino = "web scraping"  # ← Cambia este valor

url_api = f"http://export.arxiv.org/api/query?search_query=all:{termino}&start=0&max_results=5&sortBy=relevance"
respuesta = requests.get(url_api)
soup_xml = BeautifulSoup(respuesta.text, "xml")

for i, entry in enumerate(soup_xml.find_all("entry"), 1):
    titulo = entry.find("title").text.strip().replace("\n", " ")
    fecha = entry.find("published").text[:10]
    print(f"{i}. [{fecha}] {titulo}")

---
## Parte 3: API de Semantic Scholar

### ¿Qué es Semantic Scholar?

**Semantic Scholar** (semanticscholar.org) es un buscador académico creado por el Allen Institute for AI. Indexa más de 200 millones de papers de todas las disciplinas.

### ¿Por qué usar su API en vez de scraping?

Semantic Scholar ofrece una **API REST gratuita** que devuelve datos en **JSON**. Cuando una web ofrece API, es mejor usarla que hacer scraping porque:

- Los datos vienen **limpios y estructurados**
- No se rompe si cambian el diseño de la web
- Es el método que la web quiere que uses
- No necesitas parsear HTML

> Aquí ya no necesitamos `bs4` porque el JSON se parsea directamente con Python. Es importante saber cuándo **NO** hace falta scraping.

Documentación: https://api.semanticscholar.org/

### 3.1 Buscar papers por palabra clave

In [ ]:
# URL base de la API de Semantic Scholar
BASE_URL = "https://api.semanticscholar.org/graph/v1"

# Buscamos papers sobre "web scraping"
termino = "web scraping"
campos = "title,year,authors,citationCount,openAccessPdf,externalIds"

url = f"{BASE_URL}/paper/search"
params = {
    "query": termino,
    "limit": 10,
    "fields": campos,
}

respuesta = requests.get(url, params=params)
print(f"Status code: {respuesta.status_code}")
print(f"URL final: {respuesta.url}")

In [ ]:
# La respuesta es JSON → lo convertimos a diccionario de Python
datos = respuesta.json()

# Veamos las claves del resultado
print("Claves del JSON:", list(datos.keys()))
print(f"Total de resultados: {datos.get('total', 'N/A')}")
print(f"Papers devueltos: {len(datos.get('data', []))}")

In [ ]:
# Veamos la estructura del primer resultado
import json
print(json.dumps(datos["data"][0], indent=2, ensure_ascii=False))

In [ ]:
# Extraemos los datos relevantes de cada paper
resultados_ss = []

for paper in datos["data"]:
    # Nombres de los autores
    autores = [a["name"] for a in paper.get("authors", [])]
    
    # Enlace al PDF (si existe acceso abierto)
    pdf_info = paper.get("openAccessPdf")
    pdf_url = pdf_info["url"] if pdf_info else "No disponible"
    
    # DOI (si existe)
    external = paper.get("externalIds", {})
    doi = external.get("DOI", "")
    
    resultados_ss.append({
        "titulo": paper["title"],
        "año": paper.get("year", ""),
        "primer_autor": autores[0] if autores else "",
        "num_autores": len(autores),
        "citas": paper.get("citationCount", 0),
        "doi": doi,
        "pdf": pdf_url,
    })

print(f"{len(resultados_ss)} papers procesados")

In [ ]:
df_ss = pd.DataFrame(resultados_ss)

# Ordenamos por número de citas (los más citados primero)
df_ss = df_ss.sort_values("citas", ascending=False).reset_index(drop=True)
df_ss

### 3.2 Obtener detalles de un paper concreto

Si conocemos el ID de un paper (DOI, arXiv ID, etc.), podemos pedir más detalles.

In [ ]:
# Pedimos detalles del paper "Attention Is All You Need" usando su arXiv ID
paper_id = "arXiv:1706.03762"
campos = "title,year,authors,abstract,citationCount,influentialCitationCount,tldr"

url = f"{BASE_URL}/paper/{paper_id}"
respuesta = requests.get(url, params={"fields": campos})

paper = respuesta.json()

print(f"Título: {paper['title']}")
print(f"Año: {paper['year']}")
print(f"Autores: {len(paper['authors'])}")
print(f"Citas totales: {paper['citationCount']:,}")
print(f"Citas influyentes: {paper['influentialCitationCount']:,}")

# TLDR = resumen automático generado por Semantic Scholar
tldr = paper.get("tldr")
if tldr:
    print(f"\nTLDR: {tldr['text']}")

### 3.3 Buscar papers de un autor

También podemos buscar por nombre de autor y ver sus publicaciones.

In [ ]:
# Buscamos al autor
nombre_autor = "Yoshua Bengio"

url = f"{BASE_URL}/author/search"
respuesta = requests.get(url, params={"query": nombre_autor, "limit": 1})
datos_autor = respuesta.json()

# Obtenemos su ID
autor_id = datos_autor["data"][0]["authorId"]
autor_nombre = datos_autor["data"][0]["name"]
print(f"Autor encontrado: {autor_nombre} (ID: {autor_id})")

In [ ]:
# Ahora pedimos sus papers más citados
url = f"{BASE_URL}/author/{autor_id}/papers"
params = {
    "fields": "title,year,citationCount",
    "limit": 10,
}

respuesta = requests.get(url, params=params)
papers_autor = respuesta.json()["data"]

# Convertimos a DataFrame y ordenamos por citas
df_autor = pd.DataFrame(papers_autor)
df_autor = df_autor.sort_values("citationCount", ascending=False).reset_index(drop=True)
df_autor[["title", "year", "citationCount"]]

---
## Resumen y comparación

| | Scraping HTML | API XML (arXiv) | API REST (Semantic Scholar) |
|---|---|---|---|
| **Herramientas** | `requests` + `bs4` | `requests` + `bs4` (parser XML) | `requests` + `json` |
| **Formato respuesta** | HTML | XML (Atom) | JSON |
| **Fragilidad** | Alta (si cambia el diseño, se rompe) | Baja (formato estable) | Baja (formato estable) |
| **Velocidad** | Hay que parsear mucho HTML | Rápido | Muy rápido |
| **Cuándo usarlo** | Cuando NO hay API | Cuando hay API que devuelve XML | Cuando hay API REST moderna |
| **¿Necesita bs4?** | Sí | Sí (parser XML) | No |
